# Ejercicio 6: Regresión Velocidad del Viento

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import warnings
warnings.filterwarnings('ignore') # setting ignore as a parameter
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

***Nota:*** *En caso de visualizar el mensaje 'ModuleNotFoundError', ignorarlo.*

### Contexto


Considere el conjunto de datos Wind Speed. Implemente la
versión de regresión de cada uno de los modelos estudiados en clases, para predecir velocidad del viento horaria (VENTO, VELOCIDADE HORARIA (m/s)) en el conjunto de datos suministrado. Construir una tabla de error con las métricas usuales de regresión, MAPE, RMSE, R2 (ver Table 2). Realice particiones de entrenamiento y validación, con base en lo descrito en la Figura 1. Estas particiones siguen la tendencia de la velocidad del viento. Utilice la métrica RMSE en la evaluación y validación, para seleccionar el mejor modelo de regresión. El pliegue de validación en cada partición, debe estar siempre ubicado en el porcentaje final de cada partición, debido a que el tiempo es fundamental en dichas predicciones.


In [39]:
# lectura de datos
viento_df = pd.read_csv('C:/Users/kaes1/Downloads/data_treino_dv_df_2000_2010.csv')
viento_df.head()

,HORA (UTC),"VENTO, DIREï¿½ï¿½O HORARIA (gr) (ï¿½ (gr))","VENTO, VELOCIDADE HORARIA (m/s)",UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),TEMPERATURA Mï¿½XIMA NA HORA ANT. (AUT) (ï¿½C),TEMPERATURA Mï¿½NIMA NA HORA ANT. (AUT) (ï¿½C),"UMIDADE RELATIVA DO AR, HORARIA (%)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)","PRECIPITAï¿½ï¿½O TOTAL, HORï¿½RIO (mm)","VENTO, RAJADA MAXIMA (m/s)",PRESSï¿½O ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSï¿½O ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)
0,12:00,0.809017,1.8,69.0,60.0,22.6,20.7,61.0,888.2,0.0,3.8,888.2,887.7
1,13:00,0.965926,2.7,62.0,55.0,24.2,22.5,55.0,888.4,0.0,4.7,888.4,888.2
2,14:00,0.891007,2.0,56.0,50.0,25.5,24.3,51.0,888.1,0.0,4.9,888.4,888.1
3,15:00,0.848048,2.5,52.0,44.0,27.4,25.0,44.0,887.4,0.0,5.8,888.1,887.4
4,16:00,0.224951,2.4,50.0,43.0,27.1,25.5,46.0,886.5,0.0,5.8,887.4,886.5


In [40]:
# Nombres de columnas
viento_df.columns

Index(['HORA (UTC)', 'VENTO, DIREï¿½ï¿½O HORARIA (gr) (ï¿½ (gr))',
       'VENTO, VELOCIDADE HORARIA (m/s)',
       'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
       'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)',
       'TEMPERATURA Mï¿½XIMA NA HORA ANT. (AUT) (ï¿½C)',
       'TEMPERATURA Mï¿½NIMA NA HORA ANT. (AUT) (ï¿½C)',
       'UMIDADE RELATIVA DO AR, HORARIA (%)',
       'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)',
       'PRECIPITAï¿½ï¿½O TOTAL, HORï¿½RIO (mm)', 'VENTO, RAJADA MAXIMA (m/s)',
       'PRESSï¿½O ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)',
       'PRESSï¿½O ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)'],
      dtype='object')

Por dificultad de compresión de los nombres, se realizará el cambio de nombres para los campos del dataset

* **Renombrar variables**

In [41]:
# Renombrar variables
nuevos_nombres = {'HORA (UTC)': 'HORA (UTC)', 
                  'VENTO, DIREï¿½ï¿½O HORARIA (gr) (ï¿½ (gr))': 'DirVientoHoraria',
                  'VENTO, VELOCIDADE HORARIA (m/s)': 'VelHorariaViento',
                  'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)': 'HumRelMaxHAnt',
                  'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)': 'HumRelMinHAnt',
                  'TEMPERATURA Mï¿½XIMA NA HORA ANT. (AUT) (ï¿½C)': 'TempMaxHAnt',
                  'TEMPERATURA Mï¿½NIMA NA HORA ANT. (AUT) (ï¿½C)': 'TempMinHAnt',
                  'UMIDADE RELATIVA DO AR, HORARIA (%)': 'HumRelHoraria',
                  'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'PresionAtmNivEstHor',
                  'PRECIPITAï¿½ï¿½O TOTAL, HORï¿½RIO (mm)': 'PreciTotalHr',
                  'VENTO, RAJADA MAXIMA (m/s)': 'MaxRafagaViento',
                  'PRESSï¿½O ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)': 'PreAtmMaxHAnt',
                  'PRESSï¿½O ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)': 'PreAtmMinHAnt'}
viento_df.rename(columns=nuevos_nombres, inplace=True)
viento_df.columns


Index(['HORA (UTC)', 'DirVientoHoraria', 'VelHorariaViento', 'HumRelMaxHAnt',
       'HumRelMinHAnt', 'TempMaxHAnt', 'TempMinHAnt', 'HumRelHoraria',
       'PresionAtmNivEstHor', 'PreciTotalHr', 'MaxRafagaViento',
       'PreAtmMaxHAnt', 'PreAtmMinHAnt'],
      dtype='object')

### Preparación de los datos

Se crea un nuevo dataframe con solo las variables que se van a utilizar en el analisis:

In [42]:
v_df = viento_df[['HORA (UTC)','VelHorariaViento','DirVientoHoraria','HumRelMinHAnt', 'PreciTotalHr', 'MaxRafagaViento']]
v_df.head()

,HORA (UTC),VelHorariaViento,DirVientoHoraria,HumRelMinHAnt,PreciTotalHr,MaxRafagaViento
0,12:00,1.8,0.809017,60.0,0.0,3.8
1,13:00,2.7,0.965926,55.0,0.0,4.7
2,14:00,2.0,0.891007,50.0,0.0,4.9
3,15:00,2.5,0.848048,44.0,0.0,5.8
4,16:00,2.4,0.224951,43.0,0.0,5.8


In [43]:
v_df.shape

(87693, 6)

La varia **HORA (UTC)** nos permite saber la hora en que se registro la velocidad del viento, por lo que se necesita esta variable para predecir la velocidad del viento. Para incluirla en el modelo, se debe convertir a valores númericos.

In [44]:
def convert_to_hours(time_str):
    parts = time_str.split(':')
    minutes = int(parts[0]) * 60 + int(parts[1])
    return minutes/(60*24)

# Aplicar la función lambda a la columna 'Hora' y almacenar el resultado en una nueva columna 'Hora_Num'
v_df['Hora_Num'] = v_df['HORA (UTC)'].apply(lambda x: convert_to_hours(x))
print(v_df['Hora_Num'].unique())

[0.5        0.54166667 0.58333333 0.625      0.66666667 0.70833333
 0.75       0.79166667 0.83333333 0.875      0.91666667 0.95833333
 0.         0.04166667 0.08333333 0.125      0.16666667 0.20833333
 0.25       0.29166667 0.33333333 0.375      0.41666667 0.45833333]


Con el codigo anterior se crea una nueva variable **Hora_Num**, la cual representa en fracciones las horas del día.

In [45]:
# Eliminar variable HORA (UTC)
trainval = v_df.drop('HORA (UTC)', axis=1)
trainval.head()

,VelHorariaViento,DirVientoHoraria,HumRelMinHAnt,PreciTotalHr,MaxRafagaViento,Hora_Num
0,1.8,0.809017,60.0,0.0,3.8,0.500000
1,2.7,0.965926,55.0,0.0,4.7,0.541667
2,2.0,0.891007,50.0,0.0,4.9,0.583333
3,2.5,0.848048,44.0,0.0,5.8,0.625000
4,2.4,0.224951,43.0,0.0,5.8,0.666667


Ahora se separa la variable dependiente de las variables explicativas:

In [46]:
X = trainval.drop('VelHorariaViento', axis=1)
y = trainval['VelHorariaViento']

A continuación se genera una secuencia, la cual indicará cuantas particiones se van a realizar. Por motivos de capacidad computacional y tiempos de ejecución, solo se van a hacer 60 particiones.

In [47]:
# Secuencia de particiones
secuencia = []

valor_inicial = 48
paso = 24
num_elementos = 60

for i in range(num_elementos):
    secuencia.append(valor_inicial)
    valor_inicial += paso

print(secuencia)

[48, 72, 96, 120, 144, 168, 192, 216, 240, 264, 288, 312, 336, 360, 384, 408, 432, 456, 480, 504, 528, 552, 576, 600, 624, 648, 672, 696, 720, 744, 768, 792, 816, 840, 864, 888, 912, 936, 960, 984, 1008, 1032, 1056, 1080, 1104, 1128, 1152, 1176, 1200, 1224, 1248, 1272, 1296, 1320, 1344, 1368, 1392, 1416, 1440, 1464]


### KNN Regressor

El primer algoritmo de regresión que se va a aplicar es knn regressor. Para este caso, se va buscar cual es el valor de **k_neighbors** que permite obtener el RMSE mas bajo. Se probaran valores de k entre 1 y 20.

Como este algoritmo se basa en distancias, se vuelve sensible a la escala de los datos, por lo que se aplicará una trasnformación a los datos para trabajarlos en la misma escala. En este caso se utilizará la opción MinMaxScaler.

In [48]:
# función para calcular MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [49]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

scaler = MinMaxScaler(feature_range=(0, 1))
best_score=1000
for n_n in range(1,21,1):
    scores_rmse=[]
    # print(n_n)
    for size in secuencia:
        X_train, X_test, y_train, y_test = train_test_split(X.iloc[:size, :], y.iloc[:size,], test_size=24,shuffle = False, stratify= None)
        
        # Se aplica trasnformación a los datos
        X_train_= scaler.fit_transform(X_train, y_train)    
        reg = KNeighborsRegressor(n_neighbors=n_n)
        reg.fit(X_train_, y_train)
        # Ahora a los datos de validación
        X_test_=scaler.transform(X_test)
        y_pred = reg.predict(X_test_)
        score = np.sqrt(mean_squared_error(y_test, y_pred))
        mape = mean_absolute_percentage_error(y_test, y_pred)
        r2 = reg.score(X_test, y_test)
        scores_rmse.append(score)
    score_cv = np.mean(scores_rmse)
    # print(score_cv)
    if score_cv < best_score:
        best_score = score_cv        
        best_parameters = {'n_neighbors': n_n,}
print(best_parameters)
print(best_score)

{'n_neighbors': 7}
0.589396282026528


Para este caso, el RMSE mas bajo se obtuvo con 7 vecinos mas cercanos. Teniendo en cuenta el valor obtenido para el parámetro **n_neighbors**, se entrena el nuevo modelo con este parámetro, y considerando todos los datos. Los ultimos 24 registros se toman como set de prueba.

In [50]:
knn_metrics = {'RMSE': 1000,
               'MAPE': 0,
               'R2': 0}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=24,shuffle = False, stratify= None)
X_train_= scaler.fit_transform(X_train, y_train)    
reg = KNeighborsRegressor(**best_parameters)
reg.fit(X_train_, y_train)
# Ahora a los datos de validación
X_test_=scaler.transform(X_test)
y_pred = reg.predict(X_test_)
score = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = reg.score(X_test_, y_test)

knn_metrics['RMSE'] = score
knn_metrics['MAPE'] = mape
knn_metrics['R2'] = r2

#Resultados
print(knn_metrics)

{'RMSE': 0.7518797305764405, 'MAPE': 15.480043111557867, 'R2': 0.7441896407873396}


### Ridge

Ahora se aplicará la regresión Ridge. Para este caso, se va buscar cual es el valor de **alpha** que permite obtener el RMSE mas bajo. Se probaran los siguientes valores para alpha: [0.1, 0.25, 0.5, 0.75, 1, 2.5, 5, 7.5, 10]

In [51]:
from sklearn.linear_model import Ridge

best_score=1000
for alpha in [0.1, 0.25, 0.5, 0.75, 1, 2.5, 5, 7.5, 10]:
    scores_rmse=[]
    for size in secuencia:
        X_train, X_test, y_train, y_test = train_test_split(X.iloc[:size, :], y.iloc[:size,], test_size=24,shuffle = False, stratify= None)
        reg = Ridge(alpha=alpha)
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        score = np.sqrt(mean_squared_error(y_test, y_pred))
        mape = mean_absolute_percentage_error(y_test, y_pred)
        r2 = reg.score(X_test, y_test)
        scores_rmse.append(score)
    score_cv = np.mean(scores_rmse)
    if score_cv < best_score:
        best_score = score_cv        
        best_parameters = {'alpha': alpha,}
print(best_parameters)
print(best_score)

{'alpha': 5}
0.5479074780555206


Para este caso, el RMSE mas bajo se obtuvo con un **alpha** de 5. Teniendo en cuenta el valor obtenido para el parámetro **alpha**, se entrena el nuevo modelo con este parámetro, y considerando todos los datos. Los ultimos 24 registros se toman como set de prueba.

In [52]:
ridge_metrics = {'RMSE': 1000,
               'MAPE': 0,
               'R2': 0}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=24, shuffle = False, stratify= None)
reg = Ridge(**best_parameters)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
score = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = reg.score(X_test, y_test)
ridge_metrics['RMSE'] = score
ridge_metrics['MAPE'] = mape
ridge_metrics['R2'] = r2

# Resultados
print(ridge_metrics)

{'RMSE': 0.5461064918915628, 'MAPE': 13.700323149102891, 'R2': 0.8650490171574026}


### Lasso

Ahora se aplicará la regresión Lasso. Para este caso, se va buscar cual es el valor de **alpha** que permite obtener el RMSE mas bajo. Se probaran los siguientes valores para alpha: [0.1, 0.25, 0.5, 0.75, 1, 2.5, 5, 7.5, 10]

In [53]:
from sklearn.linear_model import Lasso

best_score=1000
for alpha in [0.1, 0.25, 0.5, 0.75, 1, 2.5, 5, 7.5, 10]:
    scores_rmse=[]
    for size in secuencia:
        X_train, X_test, y_train, y_test = train_test_split(X.iloc[:size, :], y.iloc[:size,], test_size=24,shuffle = False, stratify= None)
        reg = Lasso(alpha=alpha)
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        score = np.sqrt(mean_squared_error(y_test, y_pred))
        mape = mean_absolute_percentage_error(y_test, y_pred)
        r2 = reg.score(X_test, y_test)
        scores_rmse.append(score)
    score_cv = np.mean(scores_rmse)
    if score_cv < best_score:
        best_score = score_cv        
        best_parameters = {'alpha': alpha,}
print(best_parameters)
print(best_score)

{'alpha': 0.1}
0.5660001173780793


Para este caso, el RMSE mas bajo se obtuvo con un **alpha** de 0.1. Teniendo en cuenta el valor obtenido para el parámetro **alpha**, se entrena el nuevo modelo con este parámetro, y considerando todos los datos. Los ultimos 24 registros se toman como set de prueba.

In [54]:
lasso_metrics = {'RMSE': 1000,
               'MAPE': 0,
               'R2': 0}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=24, shuffle = False, stratify= None)
reg = Lasso(**best_parameters)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
score = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = reg.score(X_test, y_test)
lasso_metrics['RMSE'] = score
lasso_metrics['MAPE'] = mape
lasso_metrics['R2'] = r2

# Resultados
print(lasso_metrics)
print("Número de features utilizados: {}".format(np.sum(reg.coef_ != 0)))

{'RMSE': 0.5505372535799, 'MAPE': 13.609275445096984, 'R2': 0.8628503204241615}
Número de features utilizados: 3


### XGBoost

El siguiente algoritmo a evaluar será XGBoost, basado en árboles de decisión. Para este algoritmo, se van a modificar los parámetros **max_depth** y **n_estimators**, hasta encontrar la combinación de valores que permitan obtener el menor RMSE.

In [55]:
import xgboost as xgb

best_score=1000
for max_depth in [4, 5, 6]:
    for n_estimators in [500, 600, 700]:      
        scores_rmse=[]
        for size in secuencia:
            X_train, X_test, y_train, y_test = train_test_split(X.iloc[:size, :], y.iloc[:size,], test_size=24,shuffle = False, stratify= None)
            reg = xgb.XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, objective="reg:squarederror")
            reg.fit(X_train, y_train)
            y_pred = reg.predict(X_test)
            score = np.sqrt(mean_squared_error(y_test, y_pred))
            mape = mean_absolute_percentage_error(y_test, y_pred)
            r2 = reg.score(X_test, y_test)
            scores_rmse.append(score)
        score_cv = np.mean(scores_rmse)
        if score_cv < best_score:
            best_score = score_cv        
            best_parameters = {'max_depth': max_depth, 'n_estimators': n_estimators}
print(best_parameters)
print(best_score)

{'max_depth': 5, 'n_estimators': 500}
0.6081276356084306


Para este caso, el RMSE mas bajo se obtuvo con un **max_depth** de 5 y **n_estimators** con valor 500. Teniendo en cuenta los valores obtenidos para los parámetros, se entrena el nuevo modelo considerando todos los datos. Los ultimos 24 registros se toman como set de prueba.

In [56]:
xgb_metrics = {'RMSE': 1000,
               'MAPE': 0,
               'R2': 0}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=24, shuffle = False, stratify= None)
reg = xgb.XGBRegressor(**best_parameters)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
score = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = reg.score(X_test, y_test)
xgb_metrics['RMSE'] = score
xgb_metrics['MAPE'] = mape
xgb_metrics['R2'] = r2

# Resultados
print(xgb_metrics)


{'RMSE': 0.6384089999696699, 'MAPE': 12.688368472065182, 'R2': 0.8155751771395297}


### SVM

El siguiente algortimo a aplicar es SVM para regresión. En este caso se van a alternar los valores de los parámetros gamma y C, para encontrar la combinación de parámetros que permite obtener el menor RMSE. Para este modelo se aplica una trasnformación de los datos antes del entrenamiento, ya que este algoritmo es sensible a la escala de los datos de entrada.

In [57]:
from sklearn.svm import SVR

scaler = MinMaxScaler(feature_range=(0, 1))
best_score=1000
for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        scores_rmse=[]
    # print(n_n)
        for size in secuencia:
            X_train, X_test, y_train, y_test = train_test_split(X.iloc[:size, :], y.iloc[:size,], test_size=24,shuffle = False, stratify= None)
            
            # Se aplica trasnformación a los datos
            X_train_= scaler.fit_transform(X_train, y_train)    
            reg = SVR(gamma=gamma, C=C)
            reg.fit(X_train_, y_train)
            # Ahora a los datos de validación
            X_test_=scaler.transform(X_test)
            y_pred = reg.predict(X_test_)
            score = np.sqrt(mean_squared_error(y_test, y_pred))
            mape = mean_absolute_percentage_error(y_test, y_pred)
            r2 = reg.score(X_test, y_test)
            scores_rmse.append(score)
        score_cv = np.mean(scores_rmse)
        # print(score_cv)
        if score_cv < best_score:
            best_score = score_cv        
            best_parameters = {'gamma': gamma, 'C':C}
print(best_parameters)
print(best_score)

{'gamma': 0.1, 'C': 10}
0.5238636296350053


Para este algoritmo, el RMSE mas bajo se obtuvo con **gamma** de 0.1 y **C** con valor 10. Teniendo en cuenta los valores obtenidos para los parámetros, se entrena el nuevo modelo considerando todos los datos. Los ultimos 24 registros se toman como set de prueba.

In [58]:
svm_metrics = {'RMSE': 1000,
               'MAPE': 0,
               'R2': 0}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=24,shuffle = False, stratify= None)
X_train_= scaler.fit_transform(X_train, y_train)    
reg = SVR(**best_parameters)
reg.fit(X_train_, y_train)
# Ahora a los datos de validación
X_test_=scaler.transform(X_test)
y_pred = reg.predict(X_test_)
score = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = reg.score(X_test_, y_test)
svm_metrics['RMSE'] = score
svm_metrics['MAPE'] = mape
svm_metrics['R2'] = r2

# Resultados
print(svm_metrics)


{'RMSE': 0.5000957674840907, 'MAPE': 12.75748442239304, 'R2': 0.8868309341093163}


### MLP

Finalmente, se aplica el algortimo de Multilayer Perceptron, para este algoritmo se van a modificar los valores de los parámetros **hidden_layer_sizes** y **alpha**, hasta encontrar la mejor combinación para minimizar el RMSE. Para este modelo, se aplica una tranformación a los datos de entrenamientos antes de ingresarlos al modelo, para que los datos esten en la misma escala.

In [59]:
from sklearn.neural_network import MLPRegressor

scaler = MinMaxScaler(feature_range=(0, 1))
best_score=1000
for hidden_layer_sizes in [[10,10], [100,100]]:
    for alpha in [0.0001, 0.01, 0.1, 1]:
        scores_rmse=[]
        # print(n_n)
        for size in secuencia:
            X_train, X_test, y_train, y_test = train_test_split(X.iloc[:size, :], y.iloc[:size,], test_size=24,shuffle = False, stratify= None)
            
            # Se aplica trasnformación a los datos
            X_train_= scaler.fit_transform(X_train, y_train)    
            reg = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes, alpha=alpha)
            reg.fit(X_train_, y_train)
            # Ahora a los datos de validación
            X_test_=scaler.transform(X_test)
            y_pred = reg.predict(X_test_)
            score = np.sqrt(mean_squared_error(y_test, y_pred))
            mape = mean_absolute_percentage_error(y_test, y_pred)
            r2 = reg.score(X_test, y_test)
            scores_rmse.append(score)
        score_cv = np.mean(scores_rmse)
        # print(score_cv)
        if score_cv < best_score:
            best_score = score_cv        
            best_parameters = {'hidden_layer_sizes': hidden_layer_sizes, 'alpha': alpha}
print(best_parameters)
print(best_score)

{'hidden_layer_sizes': [100, 100], 'alpha': 1}
0.5413893222410517


Para este algoritmo, el RMSE mas bajo se obtuvo con **hidden_layer_sizes** con valor [100,100] y **alpha** con valor 1. Teniendo en cuenta los valores obtenidos para los parámetros, se entrena el nuevo modelo considerando todos los datos. Los ultimos 24 registros se toman como set de prueba.

In [60]:
mlp_metrics = {'RMSE': 1000,
               'MAPE': 0,
               'R2': 0}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=24,shuffle = False, stratify= None)
X_train_= scaler.fit_transform(X_train, y_train)    
reg = MLPRegressor(**best_parameters)
reg.fit(X_train_, y_train)
# Ahora a los datos de validación
X_test_=scaler.transform(X_test)
y_pred = reg.predict(X_test_)
score = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = reg.score(X_test_, y_test)
mlp_metrics['RMSE'] = score
mlp_metrics['MAPE'] = mape
mlp_metrics['R2'] = r2

# Resultados
print(mlp_metrics)

{'RMSE': 0.6924316800302449, 'MAPE': 15.819256429940562, 'R2': 0.7830422180895293}


### Resultados

Luego de realizar las iteraciones en busca de los mejores parámetros para todos los algoritmos de regresión, se obtienen finalmente las siguientes metricas de evaluación:

In [37]:
resultados = {'knn_metrics': knn_metrics,
              'ridge_metrics': ridge_metrics,
              'lasso_metrics': lasso_metrics,
              'xgb_metrics':xgb_metrics,
              'svm_metrics':svm_metrics,
              'mlp_metrics':mlp_metrics}
eval_df = pd.DataFrame(resultados)
display(eval_df)

,knn_metrics,ridge_metrics,lasso_metrics,xgb_metrics,svm_metrics,mlp_metrics
RMSE,0.751880,0.546106,0.550537,0.638409,0.500096,0.699306
MAPE,15.480043,13.700323,13.609275,12.688368,12.757484,15.935268
R2,0.744190,0.865049,0.862850,0.815575,0.886831,0.778713


Al observar la métricas obtenidas por los modelos entrenados con cada algoritmo, se puede observar que el algoritmo de regresión que entregó las mejores metricas de evaluación es **SVM (SVR)**, ya que el modelo creado con este algoritmo tiene el RMSE mas bajo entre todos los modelos, y además tiene el R2 mas alto, lo que indica que el modelo explica alrededor del 88% de la variabilidad de la varible respuesta, **Velocidad del viento horaria**.

Algunos puntos a tener en cuenta:
* Los resultados obtenidos podrian cambiar si se llegaran a evaluar los algoritmos con una mayor cantidad de particiones. Debido a limitaciones computacionales, solo se utilizaron 60 particiones de los datos.
* Se podria evaluar el metodo utilizado para escalar los datos, ya que se escogió MinMaxScaler por facilidad, pero podria haber un metodo mas adecuado para los datos utilizados.
* Se podrian considerar otros parámetros para modfiicar en los algoritmos, que puedan afectar el rendimiento de los modelos. Los escogidos durante este ejercicio son los que comunmente se modifican en la practica y la literatura, pero se podrian considerar más parámetros.